In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opencv-python mediapipe numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [3]:
import cv2
import mediapipe as mp
import math

In [4]:
def calculate_angle(a, b, c):
    angle_rad = math.acos((b.x - a.x) * (c.x - b.x) + (b.y - a.y) * (c.y - b.y))
    angle_deg = math.degrees(angle_rad)
    return angle_deg

In [18]:
input_video_path = '/content/drive/MyDrive/rope_skipping/raw videos/coach1.mp4'  # Update with your input video path
cap = cv2.VideoCapture(input_video_path)

In [19]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [20]:
output_video_path = 'output_annotated_video.avi'
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_width, frame_height))

In [21]:
avg_ankle_angle = 88.50
avg_knee_angle = 93.63
avg_toe_angle = 90.50

In [22]:
def calculate_angle(a, b, c):
    angle_rad = math.acos((b.x - a.x) * (c.x - b.x) + (b.y - a.y) * (c.y - b.y))
    angle_deg = math.degrees(angle_rad)
    return angle_deg

In [23]:
def check_ankle_angle(frame, landmarks):
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    if left_ankle and left_knee and left_hip:
        angle_ankle = calculate_angle(left_hip, left_knee, left_ankle)
        if angle_ankle < 85:
            cv2.putText(frame, "Ankle too high", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

In [24]:
def check_knee_angle(frame, landmarks):
    right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    if right_knee and right_hip:
        angle_knee = calculate_angle(right_hip, right_knee, right_ankle)
        if angle_knee < 90:
            cv2.putText(frame, "Knee too low", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

In [25]:
def check_toe_angle(frame, landmarks):
    left_toe = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]
    right_toe = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value]
    if left_toe and right_toe:
        angle_toe = calculate_angle(left_ankle, left_toe, right_toe)
        if angle_toe > 92:
            cv2.putText(frame, "Toe angle outward", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

In [26]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB (MediaPipe requires RGB input)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and get pose landmarks
    results = pose.process(rgb_frame)
    if results.pose_landmarks:
        # Extract pose landmarks
        landmarks = results.pose_landmarks.landmark

        # Check conditions and annotate frame
        check_ankle_angle(frame, landmarks)
        check_knee_angle(frame, landmarks)
        check_toe_angle(frame, landmarks)
        #check_travel_distance(frame, avg_travel_distance)
        #check_skip_time(frame, avg_skip_time)

    # Write the annotated frame to the output video
    out.write(frame)

# Release video objects
cap.release()
out.release()